## Tasca 5. Exploració de les dades

## Exercici 1

Descarrega el data set [Airlines Delay: Airline on-time statistics and delay causes](https://www.kaggle.com/giovamata/airlinedelaycauses) i carrega’l a un pandas Dataframe. Explora les dades que conté, i queda’t únicament amb les columnes que consideris rellevants.

In [1]:
# importem llibreries

import pandas as pd
import numpy as np

from datetime import datetime
import time

import os

import warnings

warnings.simplefilter('ignore')

In [2]:
# Constant per convertir de milles a Km

ML_TO_KM = 1.609344
DELAYED_MIN = 10

In [3]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

### Carreguem les dades

In [4]:
file = '../data/DelayedFlights.csv'

types = {'DepTime':str, 'CRSDepTime':str, 'ArrTime':str, 'CRSarrTime':str}

parse_dates =  parse_dates=[['Year','Month','DayofMonth','DepTime'], ['Year','Month','DayofMonth','ArrTime']]
dfdelays = pd.read_csv(file, dtype=types,  )

# Renombrem la primera columna 
dfdelays.columns = [ 'ind', *dfdelays.columns[1:]]

# importem la taula de companyies i les guardem en un diccionari

carrier_file = '../data/Unique_carriers.csv'
carriers = pd.read_csv(carrier_file, index_col=0).to_dict()['Description']

# importem els aeroports. Ho fem directament de la web enllaçada en el dataset de Kaggle

airports = pd.read_csv('http://stat-computing.org/dataexpo/2009/airports.csv', index_col=0).to_dict()['airport']

### Observem les Dades 

- **Computerized Reservations Systems (CRS).** 

[Descripció de camps](https://www.transtats.bts.gov/Fields.asp?Table_ID=236)

In [5]:
dfdelays.head()

,ind,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0000,8.0000,0,N,0,nan,nan,nan,nan,nan
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0000,10.0000,0,N,0,nan,nan,nan,nan,nan
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0000,17.0000,0,N,0,nan,nan,nan,nan,nan
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0000,10.0000,0,N,0,2.0000,0.0000,0.0000,0.0000,32.0000
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0000,10.0000,0,N,0,nan,nan,nan,nan,nan


### Descripció dels camps
- **ind**: Index 
- **Year, Month, DayofMonth, DayOfWeek**: Columnes amb l'any, mes, dia i dia de la setmana, com a **int**
- **DepTime**: Departure Time. Hora de sortida. En el csv guardat com a **float** amb un decimal  
- **CRSDepTime**: Computerized Reservations Systems Departure Time. Hora teòrica de sortida. Guardat com un **int**. 
- **ArrTime**: Arrival Time. Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers. **float
- **CRSArrTime**: CRS Arrival Time (local time: hhmm)
- **UniqueCarrier**: Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **FlightNum**: Número de vol
- **TailNum**: Número de cua. Número de l'avió
- **ActualElapsedTime**: Temps en minuts. Es una suma de _AirTime_, _TaxiIn_ i _TaxiOut_
- **CRSElapsedTime**: CRS Elapsed Time of Flight, in Minutes
- **AirTime**: Flight Time, in *Minutes*. Temps de vol **float**
- **ArrDelay**: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
- **DepDelay**: Difference in minutes between scheduled and actual departure time. Early departures show negative numbers.
- **Origin**: Aeroport d'origen.
- **Dest**: Aeroport de destí.
- **Distance**: Distance between airports (miles). 
- **TaxiIn**: Taxi In Time, in Minutes
- **TaxiOut**: Taxi Out Time, in Minutes
#### Cancel·lacions  
- **Cancelled**: Cancelled Flight Indicator (1=Yes)
- **CancellationCode**: Specifies The Reason For Cancellation:
    - A	Carrier
    - B	Weather
    - C	National Air System
    - D	Security
- **Diverted**:  Diverted Flight Indicator (1=Yes)  
#### Causes dels retràs:
- **CarrierDelay**: Carrier Delay, in Minutes
- **WeatherDelay**: Weather Delay, in Minutes
- **NASDelay**: National Air System Delay, in Minutes
- **SecurityDelay**: Security Delay, in Minutes
- **LateAircraftDelay**: Late Aircraft Delay, in Minutes

### Informació del DataFrame dfdelays

#### info() 
Número de Files, Columnes i nulls

In [6]:
dfdelays.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 30 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   ind                1936758 non-null  int64  
 1   Year               1936758 non-null  int64  
 2   Month              1936758 non-null  int64  
 3   DayofMonth         1936758 non-null  int64  
 4   DayOfWeek          1936758 non-null  int64  
 5   DepTime            1936758 non-null  object 
 6   CRSDepTime         1936758 non-null  object 
 7   ArrTime            1929648 non-null  object 
 8   CRSArrTime         1936758 non-null  int64  
 9   UniqueCarrier      1936758 non-null  object 
 10  FlightNum          1936758 non-null  int64  
 11  TailNum            1936753 non-null  object 
 12  ActualElapsedTime  1928371 non-null  float64
 13  CRSElapsedTime     1936560 non-null  float64
 14  AirTime            1928371 non-null  float64
 15  ArrDelay           1928371 non-n

In [7]:
print(f"Hi ha {dfdelays.shape[0]} registres.")
print(f'Hi ha {dfdelays.shape[1]} camps per cada registre')

Hi ha 1936758 registres.
Hi ha 30 camps per cada registre


#### describe()

In [8]:
dfdelays.describe()

,ind,Year,Month,DayofMonth,DayOfWeek,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1936758.0000,1936758.0000,1936758.0000,1936758.0000,1936758.0000,1936758.0000,1936758.0000,1928371.0000,1936560.0000,1928371.0000,...,1936758.0000,1929648.0000,1936303.0000,1936758.0000,1936758.0000,1247488.0000,1247488.0000,1247488.0000,1247488.0000,1247488.0000
mean,3341651.1513,2008.0000,6.1111,15.7535,3.9848,1634.2246,2184.2632,133.3059,134.3027,108.2771,...,765.6862,6.8130,18.2322,0.0003,0.0040,19.1794,3.7036,15.0216,0.0901,25.2965
std,2066064.9575,0.0000,3.4825,8.7763,1.9960,464.6347,1944.7023,72.0601,71.3414,68.6426,...,574.4797,5.2736,14.3385,0.0181,0.0631,43.5462,21.4929,33.8331,2.0227,42.0549
min,0.0000,2008.0000,1.0000,1.0000,1.0000,0.0000,1.0000,14.0000,-25.0000,0.0000,...,11.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,1517452.5000,2008.0000,3.0000,8.0000,2.0000,1325.0000,610.0000,80.0000,82.0000,58.0000,...,338.0000,4.0000,10.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,3242558.0000,2008.0000,6.0000,16.0000,4.0000,1705.0000,1543.0000,116.0000,116.0000,90.0000,...,606.0000,6.0000,14.0000,0.0000,0.0000,2.0000,0.0000,2.0000,0.0000,8.0000
75%,4972466.7500,2008.0000,9.0000,23.0000,6.0000,2014.0000,3422.0000,165.0000,165.0000,137.0000,...,998.0000,8.0000,21.0000,0.0000,0.0000,21.0000,0.0000,15.0000,0.0000,33.0000
max,7009727.0000,2008.0000,12.0000,31.0000,7.0000,2400.0000,9742.0000,1114.0000,660.0000,1091.0000,...,4962.0000,240.0000,422.0000,1.0000,1.0000,2436.0000,1352.0000,1357.0000,392.0000,1316.0000


#### Camps que tenen valors NaN. 

In [9]:
dfdelays.isna().sum()[dfdelays.isna().sum() > 0]

ArrTime                7110
TailNum                   5
ActualElapsedTime      8387
CRSElapsedTime          198
AirTime                8387
ArrDelay               8387
TaxiIn                 7110
TaxiOut                 455
CarrierDelay         689270
WeatherDelay         689270
NASDelay             689270
SecurityDelay        689270
LateAircraftDelay    689270
dtype: int64

>Comentaris: 
>- Hi ha **8387** vols sense ActualElapsedTime, AirTime, ArrDelay. Observem que són els mateixos vols a la següent cel·la, així com també els **7110** sense valor a l'ArrTime, TaxiIn i els 455 de TaxiOut, els 198 de CRSElapsedTime i un parell dels TailNum  
>- Hi ha **689270** registres sense dades sobre els minuts associats als retrasos (probablement perquè no tenen retrasos). De totes formes, els emplenarem amb 0 minuts.

##### Comprovem aquests camps a partir de l'ActualElapsedTime

In [10]:
act_elapsed_nans = dfdelays[dfdelays.ActualElapsedTime.isna()].isna()
act_elapsed_nans.sum()[act_elapsed_nans.sum() >0]

ArrTime              7110
TailNum                 2
ActualElapsedTime    8387
CRSElapsedTime        198
AirTime              8387
ArrDelay             8387
TaxiIn               7110
TaxiOut               455
CarrierDelay         8387
WeatherDelay         8387
NASDelay             8387
SecurityDelay        8387
LateAircraftDelay    8387
dtype: int64

##### Vols Cancel·lats o Desviats.

In [11]:
print(f'Hi ha {dfdelays.Cancelled.sum()} vols cancel·lats')
print(f'Hi ha {dfdelays.Diverted.sum()} vols desviats')

Hi ha 633 vols cancel·lats
Hi ha 7754 vols desviats


In [12]:
dfdelays[(dfdelays.Cancelled == 1) & (dfdelays.Diverted == 1)]
# No hi ha vols desviats i cancel·lats a la vegada

,ind,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay


In [13]:
dfdelays[['Diverted','Cancelled']].sum() == dfdelays[dfdelays.ActualElapsedTime.isna()][['Diverted','Cancelled']].sum()

Diverted     True
Cancelled    True
dtype: bool

> Observem que tots els NaNs a ActualElapsedTime son per vols Cancel·lats o Desviats

##### Assignem 0.0 als camps Null sobre causes de retràs

In [14]:
cols_retras = ['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']
dfdelays[cols_retras] = dfdelays[cols_retras].fillna(value=0.0)

### Decidim quins camps escollim.

In [15]:
mask = (dfdelays.Cancelled == 0) & (dfdelays.Diverted==0)
(dfdelays[mask].AirTime + dfdelays[mask].TaxiIn + dfdelays[mask].TaxiOut - dfdelays[mask].ActualElapsedTime).value_counts()

0.0000    1928371
dtype: int64

Ara mateix:
- no analitzarem les causes dels retrasos i, per això, descartarem els camps associats 
- descartem també els Números de vols, així com el Número de Cua. 
- eliminarem el DayOfWeek 
- Eliminarem els AirTime, TaxiIn, TaxiOut i ens quedarem només amb l'ActualElapsedTime. a la cel·la anterior veiem que sumen el mateix

Posteriorment, després de crear camps addicionals, eliminarem altres columnes


In [17]:
col_descartades = ['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','CancellationCode', 
                   'FlightNum','TailNum', 'AirTime','TaxiIn','TaxiOut']

dfdelays.drop(columns=col_descartades, inplace=True)

In [19]:
dfdelays.head()

,ind,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Distance,Cancelled,Diverted
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,128.0000,150.0000,-14.0000,8.0000,IAD,TPA,810,0,0
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,128.0000,145.0000,2.0000,19.0000,IAD,TPA,810,0,0
2,2,2008,1,3,4,628.0,620,804.0,750,WN,96.0000,90.0000,14.0000,8.0000,IND,BWI,515,0,0
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,90.0000,90.0000,34.0000,34.0000,IND,BWI,515,0,0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,101.0000,115.0000,11.0000,25.0000,IND,JAX,688,0,0


## Exercici 2

Fes un informe complet del data set:.

Resumeix estadísticament les columnes d’interès
Troba quantes dades faltants hi ha per columna
Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...)
Taula de les aerolínies amb més endarreriments acumulats
Quins són els vols més llargs? I els més endarrerits?
Etc.

#### Resum Columnes d'interès

In [20]:
columnes_interes = ['ArrTime','ActualElapsedTime','CRSElapsedTime','ArrDelay','DepDelay','Distance']

In [21]:
dfdelays[mask][columnes_interes].describe()

,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Distance
count,1928371.0000,1928371.0000,1928371.0000,1928371.0000,1928371.0000
mean,133.3059,134.1977,42.1999,43.0917,764.9486
std,72.0601,71.2334,56.7847,53.2660,573.8858
min,14.0000,-21.0000,-109.0000,6.0000,11.0000
25%,80.0000,82.0000,9.0000,12.0000,338.0000
50%,116.0000,116.0000,24.0000,24.0000,606.0000
75%,165.0000,165.0000,56.0000,53.0000,997.0000
max,1114.0000,660.0000,2461.0000,2467.0000,4962.0000


#### Noves Columnes

Afegim els noms dels aeroports i les companyies

In [22]:
col_descartades = []

dfdelays['carrier_name'] = dfdelays.UniqueCarrier.map(carriers)
dfdelays['Origin'] = dfdelays.Origin.map(airports)
dfdelays['Dest'] = dfdelays.Dest.map(airports)

col_descartades.append('UniqueCarrier')

> Aquí tenim un petit problema que, de moment, no arreglarem (perquè no ho necessitem), però hem de ser-ne conscients.
> Al assignar el DepTime de 2400.0 a 0.0 poso el vol a l'inici del dia. Tot i això, la resta de dades no es veuran afectades, però aquests vols, entenc, van volar al final del dia -i, per tant, la data seria posterior-

In [23]:
dfdelays.loc[dfdelays.DepTime.astype(float)==2400,'DepTime'] = '0.0'

Convertim les columnes de Any, Mes, Dia, Hora i Minut a un datetime

In [24]:

dfdelays['DepartureTime'] = dfdelays.apply(lambda x: datetime(int(x.Year), 
                                       int(x.Month), 
                                       int(x.DayofMonth), 
                                       int(x.DepTime.split('.')[0].zfill(4)[:-2]), 
                                       int(x.DepTime.split('.')[0].zfill(4)[-2:])), axis=1)

dfdelays['CRSDepartureTime'] = dfdelays.apply(lambda x: datetime(int(x.Year),
                                       int(x.Month), 
                                       int(x.DayofMonth), 
                                       int(x.CRSDepTime.zfill(4)[:-2]), 
                                       int(x.CRSDepTime.zfill(4)[-2:])), axis=1)

col_descartades.extend(['ind','Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime'])

- Cancel·lat o desviat
 **no_vol** Creem un camp per marcar els vols cancel·lats o desviats
 **endarrerit** Assignem un valor de 10 minuts endarrerit com 

In [25]:
dfdelays['no_vol'] = dfdelays.Cancelled + dfdelays.Diverted

dfdelays['endarrerit'] = dfdelays.ArrDelay > DELAYED_MIN

col_descartades.extend(['Cancelled','Diverted'])

In [26]:
dfdelays.drop(columns=col_descartades, inplace=True)

In [29]:
dfdelays.head()

,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Distance,carrier_name,DepartureTime,CRSDepartureTime,no_vol,endarrerit
0,2211.0,2225,128.0000,150.0000,-14.0000,8.0000,Washington Dulles International,Tampa International,810,Southwest Airlines Co.,2008-01-03 20:03:00,2008-01-03 19:55:00,0,False
1,1002.0,1000,128.0000,145.0000,2.0000,19.0000,Washington Dulles International,Tampa International,810,Southwest Airlines Co.,2008-01-03 07:54:00,2008-01-03 07:35:00,0,False
2,804.0,750,96.0000,90.0000,14.0000,8.0000,Indianapolis International,Baltimore-Washington International,515,Southwest Airlines Co.,2008-01-03 06:28:00,2008-01-03 06:20:00,0,True
3,1959.0,1925,90.0000,90.0000,34.0000,34.0000,Indianapolis International,Baltimore-Washington International,515,Southwest Airlines Co.,2008-01-03 18:29:00,2008-01-03 17:55:00,0,True
4,2121.0,2110,101.0000,115.0000,11.0000,25.0000,Indianapolis International,Jacksonville International,688,Southwest Airlines Co.,2008-01-03 19:40:00,2008-01-03 19:15:00,0,True


- velocitat mitjana de vol

tenim que controlar que les dades siguin adequades. Per exemple, que passa si són NaNs, o baixes?
Exemple: un vol desviat, a l'índex 317163
```python
dfdelays.iloc[317163]
```

In [31]:
# mask es un filtre per escollir els vols no cancel·lats

mask = dfdelays.no_vol == 0
(dfdelays[mask].Distance / (dfdelays[mask].CRSElapsedTime /60)).sort_values(ascending=False)

693851    45840.0000
1581586    7410.0000
163379     2587.1429
1455778    1297.0000
1170371     822.5000
             ...    
1456273      30.0000
47146        24.0000
774654       11.0000
1310766      11.0000
437861     -220.0000
Length: 1928371, dtype: float64

In [33]:
mask = dfdelays.no_vol == 0
(dfdelays[mask].Distance / (dfdelays[mask].ActualElapsedTime.astype(int) /60)).sort_values(ascending=False)

1110539   1662.8571
1129417   1242.0000
49719     1000.0000
331502     980.0000
25563      972.0000
             ...   
506245      16.2097
1873232     15.5212
774654      13.7500
1456273     13.5338
1310766     11.0000
Length: 1928371, dtype: float64

Exemples de casos amb dades extranyes. 
Alternatives amb aquests casos:
- esborrar-los (opció fàcil, però s'ha de considerar el número i si poden ser importants per la tasca a considerar)
- corregir-los manualment (possible fer-ho si son molt pocs)
- imputar un valor en base a diversos criteris automàticament

In [35]:
dfdelays.iloc[693851]

ArrTime                                 50.0
CRSArrTime                              2046
ActualElapsedTime                   107.0000
CRSElapsedTime                        1.0000
ArrDelay                            244.0000
DepDelay                            138.0000
Origin               Nashville International
Dest                               LaGuardia
Distance                                 764
carrier_name          American Airlines Inc.
DepartureTime            2008-04-27 22:03:00
CRSDepartureTime         2008-04-27 19:45:00
no_vol                                     0
endarrerit                              True
Name: 693851, dtype: object

Vols que tenen temps teòric de vol extranys, ```<10``` minuts

In [39]:
dfdelays[mask][dfdelays.CRSElapsedTime < 10]

,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Distance,carrier_name,DepartureTime,CRSDepartureTime,no_vol,endarrerit
437861,1910.0,1735,31.0000,-21.0000,95.0000,43.0000,Monterey Peninsula,San Francisco International,77,SkyWest Airlines Inc.,2008-03-08 18:39:00,2008-03-08 17:56:00,0,True
693851,50.0,2046,107.0000,1.0000,244.0000,138.0000,Nashville International,LaGuardia,764,American Airlines Inc.,2008-04-27 22:03:00,2008-04-27 19:45:00,0,True
759223,1008.0,929,27.0000,8.0000,39.0000,20.0000,Long Beach (Daugherty ),Ontario International,35,ExpressJet Airlines Inc.,2008-05-15 09:41:00,2008-05-15 09:21:00,0,True
1111660,1814.0,1704,34.0000,9.0000,70.0000,45.0000,Pueblo Memorial,City of Colorado Springs Muni,37,ExpressJet Airlines Inc.,2008-07-08 17:40:00,2008-07-08 16:55:00,0,True
1111744,1449.0,1331,78.0000,9.0000,78.0000,9.0000,Pueblo Memorial,City of Colorado Springs Muni,37,ExpressJet Airlines Inc.,2008-07-14 13:31:00,2008-07-14 13:22:00,0,True
1581586,1252.0,1127,72.0000,2.0000,85.0000,15.0000,William B Hartsfield-Atlanta Intl,Panama City-Bay County International,247,ExpressJet Airlines LLC,2008-10-20 12:40:00,2008-10-20 12:25:00,0,True


- Aerolinies amb més endarreriments?

In [51]:
# No considerem els vols cancel·lats o desviats
filtre = (dfdelays.no_vol == 0)

agrupat_carrier = dfdelays[filtre].groupby(['carrier_name']).agg({'ArrDelay':['sum','count'], 'endarrerit':'sum', 'Distance':'sum'})

agrupat_carrier.columns = ['_'.join(col).strip('_') for col in agrupat_carrier.columns.values]
# group.droplevel(0, axis=1)
agrupat_carrier.reset_index(inplace=True)
agrupat_carrier.sort_values(('endarrerit_sum'),ascending=False).head(10)


,carrier_name,ArrDelay_sum,ArrDelay_count,endarrerit_sum,Distance_sum
17,Southwest Airlines Co.,11319092.0000,376201,235202,244698861
3,American Airlines Inc.,8889066.0000,190910,144326,206431212
7,Envoy Air,6396704.0000,141223,107440,61146137
19,United Air Lines Inc.,6733013.0000,140904,103644,152029654
16,SkyWest Airlines Inc.,5978936.0000,131780,98693,58023344
5,Delta Air Lines Inc.,4535644.0000,113728,80964,110001518
8,ExpressJet Airlines Inc.,5176042.0000,103147,78577,61741939
18,US Airways Inc.,3571867.0000,98007,66264,94668681
4,Continental Air Lines Inc.,4045932.0000,99731,65637,123568066
9,ExpressJet Airlines LLC,3888131.0000,81762,62458,37730644


In [52]:
dfdelays.groupby(['carrier_name'])['endarrerit'].sum().sort_values(ascending=False).head(10)

carrier_name
Southwest Airlines Co.        235202
American Airlines Inc.        144326
Envoy Air                     107440
United Air Lines Inc.         103644
SkyWest Airlines Inc.          98693
Delta Air Lines Inc.           80964
ExpressJet Airlines Inc.       78577
US Airways Inc.                66264
Continental Air Lines Inc.     65637
ExpressJet Airlines LLC        62458
Name: endarrerit, dtype: int64

- I amb més endarreriments acumulats?

In [55]:
agrupat_carrier.sort_values('ArrDelay_sum', ascending=False)[['carrier_name','ArrDelay_sum']].head(10)

,carrier_name,ArrDelay_sum
17,Southwest Airlines Co.,11319092.0000
3,American Airlines Inc.,8889066.0000
19,United Air Lines Inc.,6733013.0000
7,Envoy Air,6396704.0000
16,SkyWest Airlines Inc.,5978936.0000
8,ExpressJet Airlines Inc.,5176042.0000
5,Delta Air Lines Inc.,4535644.0000
4,Continental Air Lines Inc.,4045932.0000
9,ExpressJet Airlines LLC,3888131.0000
13,Mesa Airlines Inc.,3691461.0000


- amb min/distancia?

In [61]:
agrupat_carrier['min_distancia'] = (agrupat_carrier[('ArrDelay_sum')] / agrupat_carrier[('Distance_sum')])
agrupat_carrier.sort_values('min_distancia',ascending=False)[['carrier_name','min_distancia']].head(10)

,carrier_name,min_distancia
13,Mesa Airlines Inc.,0.1359
7,Envoy Air,0.1046
9,ExpressJet Airlines LLC,0.1030
16,SkyWest Airlines Inc.,0.1030
6,Endeavor Air Inc.,0.1012
15,PSA Airlines Inc.,0.0995
8,ExpressJet Airlines Inc.,0.0838
0,AirTran Airways Corporation,0.0597
14,Northwest Airlines Inc.,0.0533
12,JetBlue Airways,0.0500


- % vols amb retràs

In [62]:
agrupat_carrier['perc_retras'] = (agrupat_carrier['endarrerit_sum'] / agrupat_carrier['ArrDelay_count']) * 100
agrupat_carrier.sort_values('perc_retras', ascending=False)[['carrier_name','perc_retras']].head(10)

,carrier_name,perc_retras
13,Mesa Airlines Inc.,82.5922
15,PSA Airlines Inc.,80.9353
14,Northwest Airlines Inc.,76.7335
9,ExpressJet Airlines LLC,76.3900
8,ExpressJet Airlines Inc.,76.1796
7,Envoy Air,76.0783
3,American Airlines Inc.,75.5990
6,Endeavor Air Inc.,75.5085
16,SkyWest Airlines Inc.,74.8922
12,JetBlue Airways,74.8239


- companyies amb % de vols retrasats més baixos

In [63]:
agrupat_carrier.sort_values('perc_retras', ascending=True)[['carrier_name','perc_retras']].head(10)

,carrier_name,perc_retras
2,Aloha Airlines Inc.,59.5430
17,Southwest Airlines Co.,62.5203
4,Continental Air Lines Inc.,65.8140
10,Frontier Airlines Inc.,66.9005
18,US Airways Inc.,67.6115
1,Alaska Airlines Inc.,69.3079
11,Hawaiian Airlines Inc.,70.8378
5,Delta Air Lines Inc.,71.1909
19,United Air Lines Inc.,73.5565
0,AirTran Airways Corporation,73.9238


- Quins son els vols més llargs?

In [65]:
vols_llargs = dfdelays.groupby(['Origin','Dest','carrier_name'])['Distance'].first()
vols_llargs = vols_llargs.sort_values( ascending=False).reset_index()

vols_llargs.head(10)


,Origin,Dest,carrier_name,Distance
0,Newark Intl,Honolulu International,Continental Air Lines Inc.,4962
1,Honolulu International,Newark Intl,Continental Air Lines Inc.,4962
2,Honolulu International,William B Hartsfield-Atlanta Intl,Delta Air Lines Inc.,4502
3,William B Hartsfield-Atlanta Intl,Honolulu International,Delta Air Lines Inc.,4502
4,Honolulu International,Chicago O'Hare International,American Airlines Inc.,4243
5,Honolulu International,Chicago O'Hare International,United Air Lines Inc.,4243
6,Chicago O'Hare International,Honolulu International,United Air Lines Inc.,4243
7,Chicago O'Hare International,Honolulu International,American Airlines Inc.,4243
8,Kona International At Keahole,Chicago O'Hare International,United Air Lines Inc.,4213
9,Chicago O'Hare International,Kahului,United Air Lines Inc.,4184


- Llista dels més endarrerits (top-10)

In [67]:
dfdelays.sort_values(by='ArrDelay', ascending=False).head(10)

,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Distance,carrier_name,DepartureTime,CRSDepartureTime,no_vol,endarrerit
322516,2256.0,555,459.0000,455.0000,2461.0000,2457.0000,Honolulu International,Minneapolis-St Paul Intl,3972,Northwest Airlines Inc.,2008-02-03 11:17:00,2008-02-03 18:20:00,0,True
686014,858.0,1605,154.0000,168.0000,2453.0000,2467.0000,Charlotte/Douglas International,Minneapolis-St Paul Intl,930,Northwest Airlines Inc.,2008-04-10 07:24:00,2008-04-10 14:17:00,0,True
839306,254.0,1823,172.0000,173.0000,1951.0000,1952.0000,Southwest Florida International,Detroit Metropolitan-Wayne County,1085,Northwest Airlines Inc.,2008-05-06 00:02:00,2008-05-06 15:30:00,0,True
1009553,2252.0,1825,72.0000,75.0000,1707.0000,1710.0000,Adams,Dallas-Fort Worth International,304,Envoy Air,2008-06-20 21:40:00,2008-06-20 17:10:00,0,True
1881639,1921.0,1546,259.0000,201.0000,1655.0000,1597.0000,Gen Edw L Logan Intl,Minneapolis-St Paul Intl,1124,Northwest Airlines Inc.,2008-12-19 16:02:00,2008-12-19 13:25:00,0,True
1497823,1048.0,825,102.0000,71.0000,1583.0000,1552.0000,Eppley Airfield,Minneapolis-St Paul Intl,282,Northwest Airlines Inc.,2008-09-01 09:06:00,2008-09-01 07:14:00,0,True
685437,1942.0,1800,84.0000,87.0000,1542.0000,1545.0000,Minot International,Minneapolis-St Paul Intl,449,Northwest Airlines Inc.,2008-04-27 18:18:00,2008-04-27 16:33:00,0,True
163379,1452.0,1327,292.0000,42.0000,1525.0000,1275.0000,Eagle County Regional,Miami International,1811,American Airlines Inc.,2008-01-05 08:00:00,2008-01-05 10:45:00,0,True
1214839,1325.0,1215,185.0000,193.0000,1510.0000,1518.0000,Seattle-Tacoma Intl,Minneapolis-St Paul Intl,1399,Northwest Airlines Inc.,2008-07-16 08:20:00,2008-07-16 07:02:00,0,True
521096,811.0,721,66.0000,66.0000,1490.0000,1490.0000,Nashville International,Memphis International,200,Northwest Airlines Inc.,2008-03-21 07:05:00,2008-03-21 06:15:00,0,True


## Exercici 3

#### Exporta el data set net i amb les noves columnes a Excel.

In [72]:

dfdelays.to_csv('../Data/delayedflights.csv', index=False)

In [73]:
dfdelays.to_pickle('../Data/delayedflights.pickle')

In [74]:
agrupat_carrier.to_csv('../Data/carrier_agregats.csv')